### Model 1B Replication


Original Created on Tue Feb  9 13:23:30 2016 @author: nickbecker

Replication April 24, 2018, Judy Yang



In [7]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
import scipy
import csv
from sklearn import linear_model
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.preprocessing import normalize, Imputer
from sklearn.metrics import mean_squared_error
from math import sqrt

In [14]:
#Reading ACS from the Engima website, this takes a few minutes

url='http://enigma-public.s3.amazonaws.com/projects/smoke-alarm-risk/data/acs.csv'
response = urlopen(url)
acs =pd.read_csv(url)

In [15]:
acs.shape

(578901, 250)

In [20]:
acs.columns

Index(['geoid', 'tenure_renter_occupied', 'tenure_owner_occupied',
       'vacancy_for_seasonal_recreational', 'vacancy_for_rent',
       'vacancy_for_sale_only', 'vacancy_other_vacant',
       'vacancy_rented_not_occupied', 'vacancy_for_migrant_workers',
       'vacancy_sold_not_occupied',
       ...
       'hhmove_moved_in_1990_to_1999', 'hhmove_moved_in_1969_or_earlier',
       'hhmove_moved_in_2000_to_2009', 'hhmove_moved_in_1970_to_1979',
       'hhmove_moved_in_1980_to_1989', 'hhmove_moved_in_2010_or_later',
       'qfs1_yes', 'qfs1_no', 'hdsb_no', 'hdsb_yes'],
      dtype='object', length=250)

In [21]:
#Read in Home Fire NFIRS data
tract_data = pd.read_csv('../Data/2009_2013_alarm_presence_by_tract.csv')

del tract_data['Unnamed: 0']

In [22]:

# Get the tract id from the geoid
x = acs['geoid'][100]
parsed_id = x.split('US')[1]
sum_level = x.split('US')[0]


In [23]:
acs['geoid_parsed'] = [geo[1] for geo in acs['geoid'].str.split('US')]
acs['sum_level'] = [geo[0] for geo in acs['geoid'].str.split('US')]

# understanding census coding: https://www.census.gov/geo/reference/geoidentifiers.html
# sumlevel == 15000 is block group
# sumlevel == 14000 is census tract
# sumlevel == 05000 is county

# pulling out state and county for ease of use
#acs['state'] = acs['geoid_parsed'].str[0:2]
#acs['cnty'] = acs['geoid_parsed'].str[2:5]
#acs['raw_tract'] = acs['geoid_parsed'].str[5:]

#acsC = acs.query("sum_level == '05000' ")
acsCT = acs.query("sum_level == '14000'")

acsCT['geoid'].tail()
#acsCT['state'].tail()
#acsCT['cnty'].tail()
#acsCT['raw_tract'].tail()
acsCT['geoid_parsed'].tail()
# Only need to keep the tract level, so clear some space in memory
#del acs


578840    53033027300
578841    53057952100
578842    53063012300
578866    55007960200
578867    55079017000
Name: geoid_parsed, dtype: object

In [26]:
########################################
# MERGING PREDICTORS & TARGET VARIABLE:
########################################

acsCT = acsCT.rename(columns = {'geoid_parsed':'tractid'})



In [27]:
acsCT['tractid'] = acsCT['tractid'].astype(int)


In [28]:
acs_tractid = acsCT['tractid']



In [30]:
acsCT.query(" tractid == 56043000200 ")


,tenure_renter_occupied,tenure_owner_occupied,vacancy_for_seasonal_recreational,vacancy_for_rent,vacancy_for_sale_only,vacancy_other_vacant,vacancy_rented_not_occupied,vacancy_for_migrant_workers,vacancy_sold_not_occupied,hhsex_female,...,hhmove_moved_in_1969_or_earlier,hhmove_moved_in_2000_to_2009,hhmove_moved_in_1970_to_1979,hhmove_moved_in_1980_to_1989,hhmove_moved_in_2010_or_later,qfs1_yes,qfs1_no,hdsb_no,hdsb_yes,tractid
125396,0.201575,0.798425,0.696,0.0,0.184,0.104,0.016,0.0,0.0,0.509595,...,0.014961,0.53622,0.077165,0.13937,0.069291,0.081102,0.918898,0.037372,0.02616,56043000200


In [31]:
#JY: not sure what this code does, couldn't run

#acsCT = acsCT.drop(['sum_level', 'geoid'], axis = 1)

#acsCT.dropna()

In [32]:

########################################################
# Handle missing observations by simple mean imputation
acs_features = acsCT.drop(['tractid'], axis = 1)

col_names = acs_features.columns

imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0).fit(acs_features)
acs_features = imp.transform(acs_features)

acsCT = pd.DataFrame(acs_features)
acsCT.columns = col_names
acsCT = pd.concat([acsCT.reset_index(drop=True), acs_tractid.reset_index(drop=True)], axis = 1)
########################################################



In [33]:
tract_data['tractid'].tail()
tract_data_merged = tract_data.merge(acsCT, how='left', on='tractid')
#tract_data_merged.tail()

In [34]:
tract_data_merged.isnull().sum()


tractid                                 0
all_fire_all_years                      0
alarm_unknown_all_years                 0
alarm_presented_all_years               0
alarm_not_presented_all_years           0
ratio_no_alarm_in_all                   0
ratio_no_alarm_in_all_known          6268
tenure_renter_occupied                 15
tenure_owner_occupied                  15
vacancy_for_seasonal_recreational      15
vacancy_for_rent                       15
vacancy_for_sale_only                  15
vacancy_other_vacant                   15
vacancy_rented_not_occupied            15
vacancy_for_migrant_workers            15
vacancy_sold_not_occupied              15
hhsex_female                           15
hhsex_male                             15
built_1980_to_1989                     15
built_1960_to_1969                     15
built_2010_to_later                    15
built_1990_to_1999                     15
built_1950_to_1959                     15
built_1939_or_earlier             

In [35]:
#tract_data_merged_filtered = tract_data_merged.query(" all_fire_all_years > 25 ").dropna()
tract_data_merged_filtered = tract_data_merged.query(" all_fire_all_years > 25 ")

#for x in tract_data_merged_filtered.columns:
#    print(x)

tract_data_merged_filtered.isnull().sum()


tractid                                0
all_fire_all_years                     0
alarm_unknown_all_years                0
alarm_presented_all_years              0
alarm_not_presented_all_years          0
ratio_no_alarm_in_all                  0
ratio_no_alarm_in_all_known          167
tenure_renter_occupied                 5
tenure_owner_occupied                  5
vacancy_for_seasonal_recreational      5
vacancy_for_rent                       5
vacancy_for_sale_only                  5
vacancy_other_vacant                   5
vacancy_rented_not_occupied            5
vacancy_for_migrant_workers            5
vacancy_sold_not_occupied              5
hhsex_female                           5
hhsex_male                             5
built_1980_to_1989                     5
built_1960_to_1969                     5
built_2010_to_later                    5
built_1990_to_1999                     5
built_1950_to_1959                     5
built_1939_or_earlier                  5
built_1940_to_19

In [ ]:

'''
#### upsample at arbitrary threshold for testing
from sklearn.utils import resample
high_fires_resampled = resample(tract_data_merged_filtered.query( "all_fire_all_years >= 50" ), n_samples = 1000, random_state = 12)

tract_data_merged_filtered = tract_data_merged_filtered.append(high_fires_resampled)
'''



In [36]:
########################################
# Model Preparation
########################################


data_features = tract_data_merged_filtered.drop('ratio_no_alarm_in_all', axis = 1)
data_features = data_features.drop(['tractid', 'alarm_unknown_all_years',
                                    'alarm_presented_all_years', 'alarm_not_presented_all_years',
                                    'ratio_no_alarm_in_all_known'], axis = 1)
data_target = tract_data_merged_filtered['ratio_no_alarm_in_all']

data_features.isnull().sum()


all_fire_all_years                   0
tenure_renter_occupied               5
tenure_owner_occupied                5
vacancy_for_seasonal_recreational    5
vacancy_for_rent                     5
vacancy_for_sale_only                5
vacancy_other_vacant                 5
vacancy_rented_not_occupied          5
vacancy_for_migrant_workers          5
vacancy_sold_not_occupied            5
hhsex_female                         5
hhsex_male                           5
built_1980_to_1989                   5
built_1960_to_1969                   5
built_2010_to_later                  5
built_1990_to_1999                   5
built_1950_to_1959                   5
built_1939_or_earlier                5
built_1940_to_1949                   5
built_1970_to_1979                   5
built_2000_to_2009                   5
nunits_20_or_more                    5
nunits_3_or_4                        5
nunits_2                             5
nunits_10_to_19                      5
nunits_5_to_9            

In [37]:
# Handle missing observations by simple mean imputation
col_names = data_features.columns
imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0).fit(data_features)
data_features = imp.transform(data_features)

data_features = pd.DataFrame(data_features)
data_features.columns = col_names

data_features.isnull().sum()


all_fire_all_years                   0
tenure_renter_occupied               0
tenure_owner_occupied                0
vacancy_for_seasonal_recreational    0
vacancy_for_rent                     0
vacancy_for_sale_only                0
vacancy_other_vacant                 0
vacancy_rented_not_occupied          0
vacancy_for_migrant_workers          0
vacancy_sold_not_occupied            0
hhsex_female                         0
hhsex_male                           0
built_1980_to_1989                   0
built_1960_to_1969                   0
built_2010_to_later                  0
built_1990_to_1999                   0
built_1950_to_1959                   0
built_1939_or_earlier                0
built_1940_to_1949                   0
built_1970_to_1979                   0
built_2000_to_2009                   0
nunits_20_or_more                    0
nunits_3_or_4                        0
nunits_2                             0
nunits_10_to_19                      0
nunits_5_to_9            

In [38]:

#observation_weights = data_features.all_fire_all_years/data_features.all_fire_all_years.sum()
#observation_weights_normalized = normalize(observation_weights, norm = 'l1')
#data_features = data_features.drop(['all_fire_all_years'], axis = 1)

x_train, x_test, y_train, y_test = train_test_split(data_features,
                                                    data_target,
                                                    test_size=0.1,
                                                    random_state=12)

all_fires = x_train.all_fire_all_years

observation_weights = x_train.all_fire_all_years/x_train.all_fire_all_years.sum()
observation_weights_normalized = normalize(observation_weights, norm = 'l1').ravel()
scaled_weights = x_train.all_fire_all_years/np.max(x_train.all_fire_all_years)

x_train = x_train.drop(['all_fire_all_years'], axis = 1)
x_test = x_test.drop(['all_fire_all_years'], axis = 1)


/Users/judyyang/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [39]:
# Output to csv
x_train.to_csv('../Output/x_train.csv', header = True, index = False)
x_test.to_csv('../Output/x_test.csv', header = True, index = False)
y_train.to_csv('../Output/y_train.csv', header = True, index = False)
y_test.to_csv('../Output/y_test.csv', header = True, index = False)
all_fires.to_csv('../Output/all_fires.csv', header = True, index = False)



In [40]:
#######################################
# MODELING
########################################


clf_linear = linear_model.SGDRegressor(loss = 'squared_loss', penalty = 'none', random_state = 12)
clf_linear.fit(x_train, y_train, sample_weight=scaled_weights)
#clf_linear.fit(x_train, y_train, sample_weight=observation_weights_normalized)
#clf_linear.fit(x_train, y_train)
#clf_linear = linear_model.LogisticRegression()
#clf_linear.fit(x_train, y_train)

y_pred = clf_linear.predict(x_test)

# RMSE of simple linear model
rmse = sqrt(mean_squared_error(y_test, y_pred))
print (rmse)


0.15035632560579218


In [41]:

#### Use model to predict for every census tract in the geocoded data (68075) ####

# Clean up original full dataset
tract_data_merged_simple = tract_data_merged.drop([
    'tractid', 'alarm_unknown_all_years','alarm_presented_all_years', 
    'alarm_not_presented_all_years','ratio_no_alarm_in_all_known', 
    'ratio_no_alarm_in_all', 'all_fire_all_years'], axis = 1)                          

tract_data_merged_simple.isnull().sum()

# Handle missing observations by simple mean imputation
col_names = tract_data_merged_simple.columns
imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0).fit(tract_data_merged_simple)
tract_data_merged_simple = imp.transform(tract_data_merged_simple)

tract_data_merged_simple = pd.DataFrame(tract_data_merged_simple)
tract_data_merged_simple.columns = col_names                                    
                  
tract_data_merged_simple.isnull().sum()
                  
full_preds = clf_linear.predict(tract_data_merged_simple)

### Bound probabilities
#full_preds[full_preds < 0] = 0
#full_preds[full_preds > 1] = 1

# RMSE of simple linear model on the full dataset
#print sqrt(mean_squared_error(tract_data_merged_simple_clean.ratio_no_alarm_in_all, full_preds))
print (sqrt(mean_squared_error(tract_data_merged.ratio_no_alarm_in_all, full_preds)))



0.23830587738991438


In [43]:

#tract_data['weighted_linear_pred'] = full_preds
tract_data['weighted_linear_pred'] = full_preds
#tract_data_merged['weighted_linear_pred'] = full_preds

tract_data = tract_data.sort_values(['all_fire_all_years'], ascending = False)


tract_data['weighted_linear_pred'].describe()
print (tract_data['weighted_linear_pred'].tail())

# Output to csv
tract_data.to_csv('../Output/tract_data_weighted_linear_preds_upsampled.csv', header = True, index = False)


4883     0.063434
39673    0.095473
27668    0.024855
4879     0.601956
15018    0.090759
Name: weighted_linear_pred, dtype: float64


In [45]:
#### Use model to predict for every census tract in the census data (74,001) ####

full_preds_acsCT = clf_linear.predict(acs_features)

### Bound probabilities
#full_preds[full_preds < 0] = 0
#full_preds[full_preds > 1] = 1


# Can't compute RMSE because we have no target for some of the tracts.

tract_preds_74k = pd.concat([acs_tractid.reset_index(drop=True), pd.Series(full_preds_acsCT)], axis = 1)

tract_preds_74k.columns = ['tractid', 'weighted_linear_pred']

tract_preds_74k.weighted_linear_pred.describe()


# Output to csv
tract_preds_74k.to_csv('../Output/tracts_74k_weighted_linear_preds_upsampled.csv', header = True, index = False)



